# CRORIS Projekti

In [4]:
#imports
import requests
import pandas as pd
import json
import time
import datetime

In [109]:
#make an API call to CRORIS
def get_projekt(projekt_id, end):
    api_url = "https://croris.hr/projekti-api/projekt/"+str(projekt_id)
    err = None
    try:
        response = requests.get(api_url, timeout=10).json()
    except requests.RequestException as e:
        raise e
    else:
        try:
            err = response['errorMessage'] #this key is only present if API error occurs due to the wrong string/id sent
        except KeyError:
            print('Success!') #if errorMessage key is not present that is an obvious success
        else:
            print(err+' - '+str(end))
    return response, err

In [81]:
def populate_dfs(response, df_response):
    projekt_id = response['id']
    created = datetime.datetime.now()

    df_response = pd.concat([df_response, pd.DataFrame(data = {
        'id': projekt_id,
        'response': str(response),
        'created': created},
        index=[0])]
    )
    
    return df_response

In [82]:
df_response = pd.DataFrame(columns=['id','response','created'])

In [110]:
# MAIN
err = None
end = 1

try:
    last_record = df_response['id'].iloc[-1]
except:
    last_record = 1
    
for id in range(last_record,10001):
    # set projekt ID
    projekt_id = id
    print(id)

    # cal function to get the response from Croris API
    response, err = get_projekt(projekt_id, end)
    time.sleep(1) #give it a second to rest for df to catch up

    if end == 10:
        break
    elif err == "Ne postoji traženi resurs.":
        id += 1
        end += 1
    else:
        # call function populate_dfs() and assign returned dfs to our pandas dfs
        df_response = populate_dfs(response, df_response)

print("Finally finished!")

timestr = time.strftime("%Y%m%d-%H%M%S")
drive_path = 'data2/'
# drop duplicates from all pandas dfs
df_response = df_response.drop_duplicates(subset='id', keep='last')
df_response.to_csv(drive_path+'CRORIS_df_response_'+timestr+'.csv')

7899
Success!
7900
Ne postoji traženi resurs. - 1
7901
Ne postoji traženi resurs. - 2
7902
Ne postoji traženi resurs. - 3
7903
Ne postoji traženi resurs. - 4
7904
Ne postoji traženi resurs. - 5
7905
Ne postoji traženi resurs. - 6
7906
Ne postoji traženi resurs. - 7
7907
Ne postoji traženi resurs. - 8
7908
Ne postoji traženi resurs. - 9
7909
Ne postoji traženi resurs. - 10
Finally finished!


In [105]:
df_response

,id,response,created
0,1,"{'id': 1, 'projektPoirotId': 1, 'hrSifraProjek...",2022-09-24 21:35:38.773272
0,2,"{'id': 2, 'projektPoirotId': 2, 'hrSifraProjek...",2022-09-24 21:35:40.387466
0,3,"{'id': 3, 'projektPoirotId': 3, 'hrSifraProjek...",2022-09-24 21:35:41.921438
0,7830,"{'id': 7830, 'tipProjekta': {'id': 385, 'naziv...",2022-09-24 21:36:11.046924
0,7831,"{'id': 7831, 'hrSifraProjekta': '2017-1-FR01-K...",2022-09-24 21:36:12.546895
...,...,...,...
0,7895,"{'id': 7895, 'hrSifraProjekta': 'KK.01.2.1.02....",2022-09-24 21:41:53.020493
0,7896,"{'id': 7896, 'hrSifraProjekta': 'KK.01.1.1.07....",2022-09-24 21:41:54.519785
0,7897,"{'id': 7897, 'hrSifraProjekta': 'ffpu-14-2019-...",2022-09-24 21:41:56.071726
0,7898,"{'id': 7898, 'tipProjekta': {'id': 385, 'naziv...",2022-09-24 21:41:57.612568
